In [1]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[sicmutils "0.18.0"]])

(ns chp7
    (:refer-clojure :exclude [partial zero? + - * / ref compare =])
    (:require [sicmutils.env :refer :all]
              [sicmutils.value :as v]))
(def render (comp ->infix simplify))

#'chp7/render

- the vector field is a generalization of the directional derivative to functions on a manifold. 
- we want to generalize the directional derivatives to other objects, such as vector fields or form fields.
- There are several useful choices. Different choices will give different results for the rate of change of vector fields and form fields along integral curves, but all choices of directional derivative operators must agree when computing rates of change of functions along integral curves, i.e., when applied to functions, all firectional derivative operators give
$$ \mathcal{D}_{\mathsf{v}}(\mathsf{f}) = \mathsf{v} (\mathsf{f})$$

__the directional derivative of a vector field $\mathsf{u}$ with respect to a vector field $\mathsf{v}$. __
- $\gamma$: an integral curve of the vector field $\mathsf{v}$, parametrized by $t$
- $\mathsf{m} = \gamma (t)$
- $\mathsf{u}'$: a vector field that results from transporting the vector field $\mathsf{u}$ along $\gamma$ for a parameter increment $\delta$.
- How $\mathsf{u}$ is transported to make $\mathsf{u}'$ determines the type of derivative. 
- We formulate the method of transport by $\mathsf{u}' = F_{\delta}^{\mathsf{v}} \mathsf{u}$.
- $F_{\delta}^{\mathsf{v}} \mathsf{u}$ is a linear transformation over the reals on $\mathsf{u}$, since we care about its behavior only in an incremental region around $\delta=0$
- $g(\delta) = \mathsf{u}(\mathsf{f})(\mathsf{m}) - (F_{\delta}^{\mathsf{v}} \mathsf{u}) (\mathsf{f})(\mathsf{m})$: the comparison of the original vector field at a point with the transported vector field at that point. This is a scalar function!
- we can compute the directional derivative operator using only ordinary derivatives
$$ \mathcal{D}_{\mathsf{v}} \mathsf{u}(\mathsf{f})(\mathsf{m}) = Dg(0)$$

In [4]:
(defn F->directional-derivative
    [F]
    (fn [v]
        (fn [u]
            (fn [f]
                (fn [m]
                    (let [g (fn [delta]
                                (- ((u f) m) (((((F v) delta) u) f) m)))]
                        ((D g) 0)))))))

#'chp7/F->directional-derivative

## 7.1 Lie Derivative
### Functions
$\mathcal{L}_{\mathsf{v}} \mathsf{f} = \mathsf{v}(\mathsf{f})$
### Vector Fields
- choose transport operator $F_{\delta}^{\mathsf{v}} \mathsf{y}$ to be the pushforward of $\mathsf{y}$ along the integral curves of $\mathsf{v}$, i.e. $F_{\delta}^{\mathsf{v}} \mathsf{y} = (\phi_{\delta}^{\mathsf{v}})_* \mathsf{y}$
- $(\mathcal{L}_{\mathsf{v}} \mathsf{y}) (\mathsf{f})(\mathsf{m}) = D g(0)$, where $g(\delta) = \mathsf{y}(\mathsf{f})(\mathsf{m}) - ((\phi_{\delta}^{\mathsf{v}})_* \mathsf{y}) (\mathsf{f})(\mathsf{m})$


In [6]:
(defn phi-0
    [coordsys order]
    (fn [v]
        (fn [delta]
            (fn [m]
                ((point coordsys)
                 (series:sum (((exp (* delta v)) (chart coordsys)) m)
                             order))))))
(defn F-Lie
    [Phi]
    (fn [v]
        (fn [delta]
            (pushforward-vector ((Phi v) delta) ((Phi v) (- delta))))))
(defn Lie-directional
    [coordsys order]
    (let [Phi (phi-0 coordsys order)]
        (F->directional-derivative (F-Lie Phi))))

#'chp7/Lie-directional

- Remember, the pushforward of vector fields can be expressed as pullback and pushforward of manifold functions $(\phi_{\delta}^{\mathsf{v}})_* \mathsf{y} (\mathsf{f})(\mathsf{m}) = (\mathsf{y} (\mathsf{f} \circ \phi_{\delta}^{\mathsf{v}})) (\phi_{-\delta}^{\mathsf{v}} (\mathsf{m}))$
- In fact, when $\delta$ is small, we can approximate the pushforward of the vector field by first order approximation of pullback and pushforward fo manifold functions
    - $\mathsf{f} \circ \phi_{\delta}^{\mathsf{v}} = \mathsf{f} + \delta \mathsf{v} (\mathsf{f})$
    - by viewing $\mathsf{y} (\mathsf{f} \circ \phi_{\delta}^{\mathsf{v}})$ as a manifold function, we can approximate the pushforward of it by 
    $$
    \begin{align}
    \mathsf{y} (\mathsf{f} \circ \phi_{\delta}^{\mathsf{v}})\circ \phi_{-\delta}^{\mathsf{v}} &= \mathsf{y} (\mathsf{f} \circ \phi_{\delta}^{\mathsf{v}}) - \delta \mathsf{v} (\mathsf{y} (\mathsf{f} \circ \phi_{\delta}^{\mathsf{v}}))\\
    &= \mathsf{y} (\mathsf{f} + \delta \mathsf{v} (\mathsf{f})) - \delta \mathsf{v} (\mathsf{y} (\mathsf{f} + \delta \mathsf{v} (\mathsf{f})))
    \end{align}$$
- Now, it is not hard to see
    $$ \begin{align}
    g(\delta) &= \mathsf{y}(\mathsf{f})(\mathsf{m}) - ((\phi_{\delta}^{\mathsf{v}})_* \mathsf{y}) (\mathsf{f})(\mathsf{m})\\
    &= (-\delta\mathsf{y}(\mathsf{v}(\mathsf{f})) + \delta \mathsf{v}(\mathsf{y}(\mathsf{f}))) (\mathsf{m}) + \mathcal{O}(\delta^2)\\
    &= \delta [\mathsf{v}, \mathsf{y}] (\mathsf{f}) (\mathsf{m}) + \mathcal{O}(\delta^2)
    \end{align}$$
- the Lie derivative of a vector field $\mathsf{y}$ w.r.t. a vector field $\mathsf{v}$ is a vector field which is specified by the commutator.
$$ (\mathcal{L}_{\mathsf{v}}\mathsf{y}) (\mathsf{f}) = [\mathsf{v}, \mathsf{y}] (\mathsf{f})$$

In [8]:
;; verify the Lie derivative is the commutator in R3
(def v (literal-vector-field 'v-rect R3-rect))
(def w (literal-vector-field 'w-rect R3-rect))
(def f (literal-manifold-function 'f-rect R3-rect))
(render
 ((- ((((Lie-directional R3-rect 2) v) w) f)
     ((commutator v w) f))
  ((point R3-rect) (up 'x0 'y0 'z0))))

"0"

In [10]:
(defn Lie-derivative-vector
    [V]
    (fn [Y]
        (commutator V Y)))

#'chp7/Lie-derivative-vector

$$\begin{align}
(\mathcal{L}_{\mathsf{v}}\mathsf{y}) (\mathsf{f}) &= [\mathsf{v}, \mathsf{y}] (\mathsf{f}) \\
&= \mathsf{v}(\mathsf{y}(\mathsf{f})) - \mathsf{y}(\mathsf{v}(\mathsf{f})) \\
&= \mathsf{v}(\mathsf{y}(\mathsf{f})) - \mathsf{y}(\mathcal{L}_{\mathsf{v}}(\mathsf{f}))
\end{align}$$

- $\mathsf{v}(\mathsf{y}(\mathsf{f}))$ measures the rate of change of the manifold function $\mathsf{y}(\mathsf{f})$ along the integral curves of $\mathsf{v}$
- the change in $\mathsf{y}(\mathsf{f})$ is due to
    1. the intrinsic change in $\mathsf{y}$ along the curve
    2. the change in $\mathsf{f}$ along the curve: $\mathsf{v}(\mathsf{f})$
- The second term in the commutator subtracts this latter quantity.
- To better understand this perspective, let's analyze the changes with a small $\delta$ along the curve
    - the change in $\mathsf{y}(\mathsf{f})$ is $\mathsf{y}(\mathsf{f})(\phi_{\delta}^{\mathsf{v}}(\mathsf{m})) - \mathsf{y}(\mathsf{f})(\mathsf{m}) = \delta \mathsf{v}(\mathsf{y}(\mathsf{f}))$
    - a portion of the above change is due to the change of $\mathsf{f}$ along the curve, which is $\mathsf{y} (\mathsf{f} \circ \phi_{\delta}^{\mathsf{v}} - \mathsf{f}) (\mathsf{m}) = \delta \mathsf{y} (\mathsf{v}(\mathsf{f}))$
- our conclusion is that Lie derivative measures the intrinsic change in $\mathsf{y}$ along the integral curves of $\mathsf{v}$

### An Alternative View
- the directional derivative operators satisfy Leibniz's rule $\mathcal{D}_{\mathsf{v}} (\mathsf{gu}) = \mathsf{g} (\mathcal{D}_{\mathsf{v}} \mathsf{u}) + (\mathcal{D}_{\mathsf{v}} \mathsf{g}) \mathsf{u}$
- $\mathsf{g}$: manifold function
- $\mathsf{u}$: a vector field
- product rule of Lie derivative acting on the multiplication of manifold function and vector field
$$\begin{align}
(\mathcal{L}_{\mathsf{v}}\mathsf{gu}) (\mathsf{f})
&= [\mathsf{v}, \mathsf{gu}](\mathsf{f}) \\
&= \mathsf{v}(\mathsf{gu}(\mathsf{f})) - \mathsf{gu}(\mathsf{v}(\mathsf{f})) \\
&= \mathsf{v}(\mathsf{g})\mathsf{u}(\mathsf{f})+\mathsf{gv}(\mathsf{u}(\mathsf{f}))- \mathsf{gu}(\mathsf{v}(\mathsf{f}))\\
&= \mathsf{v}(\mathsf{g})\mathsf{u}(\mathsf{f}) + \mathsf{g}[\mathsf{v}, \mathsf{u}](\mathsf{f}) \\
&= (\mathcal{L}_{\mathsf{v}}\mathsf{g})\mathsf{u}(\mathsf{f}) + \mathsf{g}(\mathcal{L}_{\mathsf{v}}\mathsf{u}) (\mathsf{f})
\end{align}$$
- we express the vector field in terms of vector basis $\mathsf{y}(\mathsf{f}) = \sum_{i} \mathsf{y}^i \mathsf{e}_i (\mathsf{f})$
- using the product rule , we have $\mathcal{L}_{\mathsf{v}}\mathsf{y}(\mathsf{f}) = \sum_{i} \big((\mathcal{L}_{\mathsf{v}}\mathsf{y}^i)\mathsf{e}_i(\mathsf{f}) + \mathsf{y}^i(\mathcal{L}_{\mathsf{v}}\mathsf{e}_i) (\mathsf{f})\big)$
- the Lie derivative of a vector field is also a vector field, we can extract the components of $\mathcal{L}_{\mathsf{v}}\mathsf{e}_i$ using the dual basis. 
- define these components as $\Delta_j^i (\mathsf{v}) = \tilde{\mathsf{e}}^i (\mathcal{L}_{\mathsf{v}}\mathsf{e}_j) = \tilde{\mathsf{e}}^i ([\mathsf{v}, \mathsf{e}_j])$
- the Lie derivative can be written as $\mathcal{L}_{\mathsf{v}}\mathsf{y}(\mathsf{f}) = \sum_{i} \big((\mathcal{L}_{\mathsf{v}}\mathsf{y}^i) + \sum_j \Delta_j^i (\mathsf{v}) \mathsf{y}^j(\mathsf{f})\big)\mathsf{e}_i(\mathsf{f})$

### Form Fields
- Extended Leibniz rule of directional derivative operators $\mathcal{D}_{\mathsf{v}}(\omega (\mathsf{y})) = \omega (\mathcal{D}_{\mathsf{v}} \mathsf{y}) + (\mathcal{D}_{\mathsf{v}}\omega) (\mathsf{y})$
- We can then define the Lie derivative of a form field $\omega$ w.r.t. the vector field $\mathsf{v}$
- $(\mathcal{L}_{\mathsf{v}}(\omega))(\mathsf{y}) = \mathsf{v}(\omega(\mathsf{y})) - \omega(\mathcal{L}_{\mathsf{v}}\mathsf{y})$
- the first term computes the range of change of the combination $\omega(\mathsf{y})$ along the integral curve of $\mathsf{v}$.
- the second term substracts $\omega$ applied to the change in $\mathsf{y}$
- The equation can be generalized to the Lie derivative of a k-form field $\omega$
- $\mathcal{L}_{\mathsf{v}}\omega(\mathsf{w}_0, ..., \mathsf{w}_{k-1}) = \mathsf{v}(\omega(\mathsf{w}_0, ..., \mathsf{w}_{k-1})) - \sum_{i=0}^{k-1} \omega(\mathsf{w}_0, ..., \mathcal{L}_{\mathsf{v}}\mathsf{w}_i, ..., \mathsf{w}_{k-1})$

### Uniform Interpretation
- abstracting the idea of Lie derivative
- The Lie derivative of an object $\mathsf{a}$, that can apply to other objects $\mathsf{b}$, to produce manifold functions $\mathsf{a}(\mathsf{b}): \mathsf{M} \rightarrow \mathsf{R}^n$

### Properties of the Lie Derivative
- linear in arguments
    - $\mathcal{L}_{\alpha \mathsf{v} + \beta \mathsf{w}} = \alpha \mathcal{L}_{\mathsf{v}} + \beta \mathcal{L}_{\mathsf{w}}$
    - $\mathcal{L}_{\mathsf{v}} (\alpha \mathsf{a} + \beta \mathsf{b}) = \alpha \mathcal{L}_{\mathsf{v}} \mathsf{a} + \beta\mathcal{L}_{\mathsf{v}} \mathsf{b}$
- For any k-form field $\omega$ and any vector field $\mathsf{v}$, the exterior derivative commutes with the Lie derivative w.r.t to the vector field
    - $\mathcal{L}_{\mathsf{v}}(\mathsf{d}\omega) = \mathsf{d} (\mathcal{L}_{\mathsf{v}} \omega)$
    - if $\omega$ is an element of surface, then $\mathsf{d}\omega$ is an element of volume. 
    - The Lie derivative computes the rate of change of its argument under a deformation described by the vector field. 
    - The answer is the same whether we deform the surface before computing the volume or compute the volume and then deform it. 

In [11]:
;; verify exterior derivative commutes with the Lie derivative in 3-d rectangular space
;; of a general one-form field
(def X (literal-vector-field 'X-rect R3-rect))
(def Y (literal-vector-field 'Y-rect R3-rect))
(def V (literal-vector-field 'V-rect R3-rect))
(def a (literal-manifold-function 'alpha R3-rect))
(def b (literal-manifold-function 'beta R3-rect))
(def c (literal-manifold-function 'gamma R3-rect))
(def R3-rect-point 
    ((point R3-rect) (up 'x0 'y0 'z0)))
(let-coordinates
 [[x y z] R3-rect]
 (def theta
     (+ (* a dx) (* b dy) (* c dz))))
(render
 (((- ((Lie-derivative V) (d theta))
      (d ((Lie-derivative V) theta)))
   X Y)
  R3-rect-point))

"0"

In [13]:
;; of a general two-form field
(let-coordinates
 [[x y z] R3-rect]
 (def omega
     (+ (* a (wedge dy dz))
        (* b (wedge dz dx))
        (* c (wedge dx dy)))))
(def Z (literal-vector-field 'Z-rect R3-rect))
(render
 (((- ((Lie-derivative V) (d omega))
      (d ((Lie-derivative V) omega)))
   X Y Z)
  R3-rect-point))

"0"

- If $\mathsf{v}$ and $\mathsf{w}$ are two vector fields, then 
    - $[\mathcal{L}_{\mathsf{v}}, \mathcal{L}_{\mathsf{w}}] = \mathcal{L}_{[\mathsf{v}, \mathsf{w}]}$

In [15]:
;; verify using an one-form field
(render
 ((((- (commutator (Lie-derivative X) (Lie-derivative Y))
       (Lie-derivative (commutator X Y)))
    theta)
   Z)
  R3-rect-point))

"0"

In [16]:
;; verify using a two-form field
(render
 ((((- (commutator (Lie-derivative X) (Lie-derivative Y))
       (Lie-derivative (commutator X Y)))
    omega)
   Z V)
  R3-rect-point))

"0"

### Exponentiating Lie Derivatives
- The Lie derivative computes the rate of change of objects as they are advanced along integral curves. 
- The Lie derivative of an object produces another object of the same type, which allows us to iterate Lie derivatives. 
- The operator $e^{t\mathcal{L}_{\mathsf{v}}} = 1 + t \mathcal{L}_{\mathsf{v}} + \frac{t^2}{2!}\mathcal{L}_{\mathsf{v}}^2 + ...$ evolves objects along the curve by parameter $t$
- Consider the simple case of advancing the coordinate-basis vector field $\partial / \partial \mathsf{y}$ by an angle $a$ around the circle. 
- Let $\mathsf{J}_z = x \partial / \partial \mathsf{y} - y \partial / \partial \mathsf{x}$, the circular vector field

In [27]:
(let-coordinates
 [[x y] R2-rect]
 (def Jz (- (* x d:dy) (* y d:dx)))
 (def evolved-vector
   (seq 
    ((((exp (* 'a (Lie-derivative Jz)))
       d:dy)
      (literal-manifold-function 'f-rect R2-rect))
     ((point R2-rect) (up 1 0))))))

#'chp7/evolved-vector

In [30]:
(render (nth evolved-vector 0))

"∂₁f-rect(up(1, 0))"

In [31]:
(render (nth evolved-vector 1))

"a ∂₀f-rect(up(1, 0))"

In [32]:
(render (nth evolved-vector 2))

"-1/2 a² ∂₁f-rect(up(1, 0))"

In [33]:
(render (nth evolved-vector 3))

"-1/6 a³ ∂₀f-rect(up(1, 0))"

In [34]:
(render (nth evolved-vector 4))

"1/24 a⁴ ∂₁f-rect(up(1, 0))"

- By inspecting the results, we conclude that 
$$ exp \big(a \mathcal{L}_{\mathsf{x} \partial / \partial \mathsf{y} - \mathsf{y} \partial /\partial \mathsf{x}}\big) \frac{\partial} {\partial \mathsf{y}} = -\mathrm{sin}(a)\frac{\partial}{\partial \mathsf{x}} + \mathrm{cos}(a) \frac{\partial} {\partial \mathsf{y}} $$

### Interior Product
- substitute a vector field $\mathsf{v}$ into the first argument of a $p$-form field $\omega$ to produce a $p-1$-form field.
- $(i_{\mathsf{v}}\omega)(\mathsf{v}_1, ..., \mathsf{v}_{p-1}) = \omega (\mathsf{v}, \mathsf{v}_1, ..., \mathsf{v}_{p-1})$
- product rule: $\mathcal{L}_{\mathsf{v}} (i_{\mathsf{y}}\omega) = i_{\mathcal{L}_{\mathsf{v}}\mathsf{y}}\omega + i_{\mathsf{y}}(\mathcal{L}_{\mathsf{v}}\omega)$
- Cartan's formula: $\mathcal{L}_{\mathsf{v}}\omega = i_{\mathsf{v}}(\mathsf{d}\omega) + \mathsf{d}(i_{\mathsf{v}}\omega)$